# Importance Sampling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Definitions:

**Probability distribution function (PDF)**: A function which describes the likelihood of sampling a specific value.

**Expected value**: 

-The long-run average value of repetitions of a random variable

-The expected value of a function, $f(x)$, with x values drawn from a PDF, $p(x)$. 
In general, $E(f(X))=\int f(x)p(x)dx$ where x is sampled from $p(x)$ (the PDF).  

		


In [1]:
fig, ax = plt.subplots(1,2, figsize=(15,5))
x = np.random.random(1000000)
x = np.random.randn(1000000)


#*******Plotting*******
ax[0].hist(x, bins=50, ec='black', color='yellow', normed=True);
ax[0].set_title('Uniform distribution (Numpy.random.random)', fontsize=15)

ax[1].hist(x, bins=50, ec='black', color='yellow', normed=True);
ax[1].set_title('Normal distribution (Numpy.random.randn)', fontsize=15)

NameError: name 'plt' is not defined

**Our notation**: 

Function to get expectation value for:  $f(x)$  	

PDF:   $p(x)$	

## Monte Carlo

“Monte Carlo is the art of approximating and expectation by the sample mean of a function of simulated random variables.”

-Eric C. Anderson, *Lecture Notes for Stat 578C*

For a random distribution X, we find the expected value $E(f(X))=\frac{1}{N}\sum^N_{n=1}f(x_n)$

Law of large numbers: E will converge to the theoretical value for large N
To improve runtime and maintain accuracy, take the average of $E(f(X))$ over many simulations but for fewer points.


# Example 1

In [ ]:
def fn(x):
    return 10*np.exp(-5*((x-3)**4))

def fn_shifted(x):
    return 10*np.exp(-5*(x**4))

def example1():

    return